---
# Seminar 3 - Grundlagen Ablaufplanung
---





---
### Einlesen der Klassen aus erster Veranstaltung 
---

In [2]:
# files from first session
from InputData import InputData
from OutputData import OutputJob  # 从名为OutputData的文件中导入OutputJob
# Additionally
import numpy


---
### 1. Codierung und Bewertung einer Lösung
---
Im Rahmen Ihrer Werkstudententätigkeit bei Dr. Best sollen Sie die Produktionsleiterin Traudel Teufel bei der Ablaufplanung unterstützen. Aktuell wird die Fertigungsreihenfolge für die nächste Woche immer freitags festgelegt. Aufgrund der zunehmenden Komplexität fällt es Frau Teufel immer schwerer, gute Ablaufpläne zu generieren. Derzeit nutzt sie Ihre lange Erfahrung im Bereich der Zahnpastafertigung und erzeugt manuell eine geeignete Bearbeitungsreihenfolge, welche Sie an die Schichtleiter weitergibt.

In der nächsten Woche sollen 11 Aufträge abgearbeitet werden. Die Inputdaten sind Ihnen in der Datei "InputFlowshopSIST.json" gegeben. Traudel Teufel plant mit folgender Reihenfolge:
**6-5-7-4-8-3-9-2-10-1-11**

Diese Auftragsreihenfolge soll auf allen Maschinen eingehalten werden (Permutation Flow Shop). Leider kann die Produktionsleiterin nicht einschätzen, wie gut Ihre Lösung tatsächlich ist. Da sie von Ihren Programmierfähigkeiten gehört hat, bittet sie Sie, diese Lösung zu bewerten. Anschließend sollen Sie außerdem Auskunft darüber geben, wann welcher Auftrag auf welcher Maschine laut Plan bearbeitet wird.

#### a.) 
Sie überlegen, dass es sinnvoll ist, eine eigene Klasse für Lösungen anzulegen, wenn Sie im folgenden verschiedene Lösungen erzeugen wollen. Deshalb schreiben Sie als erstes eine Klasse **Solution**, welche ein Dictionary mit allen Aufträge als **OutputJobs** sowie die Fertigungsreihenfolge als Liste **Permutation** enthält. Zudem sollen im Konstruktor die Attribute Makespan, TotalTardiness und TotalWeightedTardiness angelegt und zunächst auf -1 gesetzt werden. Nachdem Sie die Klasse definiert haben, bietet es sich an, als erstes Objekt dieser Klasse die von Frau Teufel vorgeschlagene Lösung mit Hilfe der Inputdaten in der Datei "InputFlowshopSIST.json" zu erzeugen. Nennen Sie diese erste Lösung **DevilSolution**.


In [3]:
from EvaluationLogic import *
class Solution:
    
    def __init__(self, joblist, permutation):
        self.Makespan = -1
        self.TotalTardiness = -1
        self.TotalWeightedTardiness = -1
        self.Permutation = permutation

        self.OutputJobs = {}
        for jobId, job in enumerate(joblist):
            self.OutputJobs[jobId] = OutputJob(job)

    def __str__(self):
        return f'The permutation {self.Permutation} results in a Makespan of {self.Makespan}'
    
    def SetPermutation(self, permutation):
        self.Permutation = permutation



#### Führen Sie im Anschluss folgenden Code aus ####
data = InputData("InputFlowshopSIST.json")
Permutation = [x-1 for x in [6,5,7,4,8,3,9,2,10,1,11]]
DevilSolution = Solution(data.InputJobs, Permutation) 

EvaluationLogic(data).DefineStartEnd(DevilSolution)
print(DevilSolution)

The permutation [5, 4, 6, 3, 7, 2, 8, 1, 9, 0, 10] results in a Makespan of 8922


#### b.) 
Um die Qualität einer Lösung einschätzen zu können, sollten Sie als nächstes eine Bewertungsfunktion für ein gegebenes Solution Objekt schreiben. Diese Methode sollte für eine gegebene Reihenfolge (Permutation) allen Aufträgen Start- und Endzeitpunkte zuweisen unter Beachtung, dass jede Maschine nur einen Auftrag zur selben Zeit bearbeiten kann und ein Auftrag nicht gleichzeitig auf mehreren Maschinen bearbeitet werden kann. Die Rüstzeiten können Sie dabei zunächst vernachlässigen. Am Ende der Einplanung können Sie für das Solution Objekt noch das Attribut Makespan festlegen.

Da die erstellte Funktion zur Bewertung einer Lösung benötigt wird, erachten Sie es als sinnvoll, diese der Klasse EvaluationLogic anzuhängen.

Abschließend können Sie Ihre neu entwickelten Methoden testen, indem Sie die von Frau Teufel vorgeschlagene Lösung **DevilSolution** bewerten.

In [4]:
class EvaluationLogic:    
    def DefineStartEnd(self, currentSolution):    
        #####
        # schedule first job: starts when finished at previous stage
        firstJob = currentSolution.OutputJobs[currentSolution.Permutation[0]]  #currentSolution.Permutation[0] 调用solution对象的permutation定语，然后选择排在第一位的job
        firstJob.EndTimes = numpy.cumsum([firstJob.ProcessingTime(x) for x in range(len(firstJob.EndTimes))])
        firstJob.StartTimes[1:] = firstJob.EndTimes[:-1]
        #####
        # schedule further jobs: starts when finished at previous stage and the predecessor is no longer on the considered machine
        for j in range(1,len(currentSolution.Permutation)):
            currentJob = currentSolution.OutputJobs[currentSolution.Permutation[j]]
            previousJob = currentSolution.OutputJobs[currentSolution.Permutation[j-1]]
            # first machine
            currentJob.StartTimes[0] = previousJob.EndTimes[0]
            currentJob.EndTimes[0] = currentJob.StartTimes[0] + currentJob.ProcessingTime(0)
            # other machines
            for i in range(1,len(currentJob.StartTimes)):
                currentJob.StartTimes[i] = max(previousJob.EndTimes[i], currentJob.EndTimes[i-1])
                currentJob.EndTimes[i] = currentJob.StartTimes[i] + currentJob.ProcessingTime(i)
        #####
        # Save Makespan and return Solution
        currentSolution.Makespan = currentSolution.OutputJobs[currentSolution.Permutation[-1]].EndTimes[-1]

EvaluationLogic().DefineStartEnd(DevilSolution)
print(DevilSolution.Makespan)

8922


Erwarteter Output:

    8922

#### c.)
Damit alle Mitarbeiter in der Fertigung detailiert über den Ablaufplan informiert werden können, soll in einer .csv Datei für alle Aufträge aufgelistet werden, wann diese bearbeitet werden sollen. Die Tabelle soll dabei die folgenden Spalten enthalten:

|Machine  |Job      |Start_Setup |End_Setup  |Start    |End 	 |
|---------|---------|------------|-----------|---------|---------|
| 1 	  | 1 	    | 0	         | 0	     | 132 	   | 481 	 |
| 1  	  | 2 	    | 0   	     | 0	     | 0 	   | 132 	 |
| ...	  | ... 	| ...        | ... 	     | ... 	   | ...	 | 

Schreiben Sie eine Methode WriteSolToCsv(), welche eine gegebene Lösung (Instanz der Klasse Solution) in eine csv Datei schreibt. Nutzen Sie dabei das Modul **csv**. Erzeugen Sie anschließend eine Ausgabe von **DevilSolution**. Da die Ausgabe nur mit einem Solution Objekt erfolgen kann, sollten die Methode an die Klasse Solution angehangen werden.



In [5]:
import csv

def WriteSolToCsv(self, fileName):
    with open(fileName, 'w') as csvFile:
        csv_writer = csv.writer(csvFile)
        csv_writer.writerow(['Machine', 'Job', 'Start_Setup', 'End_Setup', 'Start', 'End'])

        for i in range(len(self.OutputJobs[0].Operations)):
            for job in self.OutputJobs.values():
                csv_writer.writerow([i + 1, job.JobId, job.StartSetups[i], job.EndSetups[i], job.StartTimes[i], job.EndTimes[i]])

setattr(Solution, "WriteSolToCsv", WriteSolToCsv)  # setattr(object, name, value)
DevilSolution.WriteSolToCsv("DevilSolution.csv")


#### d.)
Nachdem Sie aus der Fertigung hören, dass einige Mitarbeiter Probleme beim Verständnis der csv Ausgabe haben, machen Sie sich auf die Suche nach geeigneten Grafikmodulen. Erfreulicherweise stoßen Sie schnell auf die Methode **timeline()** aus dem Modul **plotly.express**. Zudem hatte scheinbar schon eine anderer Programmierer das gleiche Problem, weshalb Ihnen jetzt ein Python Skript vorliegt, welches Sie direkt zur grafischen Darstellung nutzen können. 

Laden Sie das Skript **Gantt.py** und nutzen Sie die Methode **ganttChart**, um DevilSolution grafisch darzustellen. 

---
### 2. Dispatching Rules zur Erstellung von Lösungen
---
Trotz aller Erfahrung ist sich Traudel Teufel unsicher bezüglich der Qualität ihrer Lösung (DevilSolution). Immerhin gibt es 39916800 mögliche Permutationen für die 11 Aufträge. Aus diesem Grund überlegen Sie, wie sich konstruktiv weitere Lösungen erzeugen lassen, um die Güte der bisherigen Lösung einzuschätzen und eventuell eine bessere Lösung zu finden. 

#### a.)
Um überhaupt ein Gefühl für die Verteilung möglicher Ablaufpläne zu erhalten, lassen sich mit der wohl einfachsten Entscheidungsregel **Random Order of Service** zunächst verschiedene Lösungen erzeugen. Das Vorgehen besteht darin, dass der nächste Auftrag immer zufällig gewählt wird. Schreiben Sie deshalb eine Methode **ROS()**, welche zufällig "x" Fertigungsreihenfolgen erzeugt und die beste zurückgibt. Der Parameter "x" soll dabei der Funktion ebenso wie ein Startwert und die Stammdaten der Aufträge übergeben werden. Bei der Generierung zufälliger Permutationen kann Ihnen das Modul Numpy sicher wieder behilflich sein.

In [6]:
import numpy as np

def ROS(jobList, x, seed):
    np.random.seed(seed)
    tmpSolution = Solution(jobList, 0)
    bestCmax = np.inf # unendless 无穷大的浮点数

    for i in range(x):
        tmpPermuation = np.random.permutation(len(jobList))
        tmpSolution.SetPermutation(tmpPermuation)

        EvaluationLogic().DefineStartEnd(tmpSolution)

        if (tmpSolution.Makespan < bestCmax):
            bestCmax = tmpSolution.Makespan
            bestPerm = tmpPermuation
    
    bestSol = Solution(jobList, bestPerm)
    EvaluationLogic().DefineStartEnd(bestSol)

    return bestSol

ROSSolution = ROS(data.InputJobs, 10, 2021)
print(ROSSolution)     

The permutation [ 3  0  4  2  5  1  7 10  9  6  8] results in a Makespan of 7812


Erwarteter Output:

Reihenfolge \[3, 0, 4, 2, 5, 1, 7, 10, 9, 6, 8\] resultiert in Makespan 7812

#### b.)
Ein Mitarbeiter aus der Fertigung weist Sie darauf hin, dass Sie bei 11 Aufträgen auch alle Permutationen berechnen könnten und anschließend die Beste bestimmen können. Sie wollen deshalb eine Funktion **checkAllPermutations()** schreiben, die Ihnen nach Überprüfung aller Reihenfolgen diejenige zurückgibt, die zum besten Makespan führt. Wie lange dauert diese Rechnung?

In [7]:
from itertools import permutations
import time

def CheckAllPermutations(jobList):
    allPerms = set(permutations(range(len(jobList))))

    tmpSolution = Solution(jobList, 0)
    bestCmax = np.inf

    for tmpPermutation in allPerms:

        tmpSolution.SetPermutation(tmpPermutation)

        EvaluationLogic().DefineStartEnd(tmpSolution)

        if (tmpSolution.Makespan < bestCmax):
            bestCmax = tmpSolution.Makespan
            bestPerm = tmpPermutation
    
    bestSol = Solution(jobList, bestPerm)
    EvaluationLogic().DefineStartEnd(bestSol)

    return bestSol

# Use function to get best Solution
time1 = time.time() 
bestSolution = CheckAllPermutations(data.InputJobs)
time2 = time.time() 

# Print Results
print("Calculation time is "+ str((time2-time1)/60) + " Minutes")
print("Best solution with Cmax "+ str(bestSolution.Cmax))
print("Best order is: ", end="")
for x in bestSolution.Permutation:
    print(x)

Calculation time is 41.01693323453267 Minutes


AttributeError: 'Solution' object has no attribute 'Cmax'

Erwarteter Output:

    Calculation time is 111.684 Minutes <br>
    Beste Lösung mit Makespan 7038 <br>
    Beste Reihenfolge ist: 7 2 0 4 6 10 3 5 8 1 9

#### c.)
Nachdem eine vollständige Enumeration selbst bei kleinen Problemen langfristig keine Alternative darstellt, schlagen Sie vor, das Problem mit statischen Einplanungsregeln zu lösen. Frau Teufel ist von dieser Idee begeistert und möchte gern, dass Sie die folgenden Regeln implementieren: <br>
* FCFS (First Come First Serve)
* SPT (Shortest Processing Time)
* LPT (Longest Processing Time) 

Schreiben Sie für jede dieser Regeln eine Methode. Die Funktionen sollen jeweils ein Objekt der Klasse **Solution** ausgeben.



In [8]:
def FirstComeFirstServe(jobList):
    tmpPermutation = [x for x in range(len(jobList))] # 排列顺序就是jobList中的顺序，先到先得
    tmpSolution = Solution(jobList, tmpPermutation)
    EvaluationLogic().DefineStartEnd(tmpSolution)

    return tmpSolution

FCFSSol = FirstComeFirstServe(data.InputJobs)
print(FCFSSol)

The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298


Erwarteter Output:

    The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298

In [9]:
def ShortestProcessingTime(jobList, allMachines = False):
    jobPool = []
    if allMachines:
        for i in range(len(jobList)):
            jobPool.append((i, sum(jobList[i].ProcessingTime(x) for x in range(len(jobList[i].Operations)))))
    else:
        for i in range(len(jobList)):
            jobPool.append((i, jobList[i].ProcessingTime(0)))
            
    jobPool.sort(key= lambda t: t[1])
    tmpPermutation = [x[0] for x in jobPool]

    tmpSolution = Solution(jobList, tmpPermutation)
    EvaluationLogic().DefineStartEnd(tmpSolution)

    return tmpSolution

SPTSol = ShortestProcessingTime(data.InputJobs)
print(SPTSol)


The permutation [2, 7, 8, 0, 3, 4, 6, 10, 1, 5, 9] results in a Makespan of 7718


Erwarteter Output:

    The permutation [2, 7, 8, 0, 3, 4, 6, 10, 1, 5, 9] results in a Makespan of 7718

In [10]:
def LongestProcessingTime(jobList, allMachines = False):
    jobPool = []
    if allMachines:
        for i in range(len(jobList)):
            jobPool.append((i, sum(jobList[i].ProcessingTime(x) for x in range(len(jobList[i].Operations)))))
    else:
        for i in range(len(jobList)):
            jobPool.append((i, jobList[i].ProcessingTime(0)))
            
    jobPool.sort(key= lambda t: -t[1])
    tmpPermutation = [x[0] for x in jobPool]

    tmpSolution = Solution(jobList, tmpPermutation)
    EvaluationLogic().DefineStartEnd(tmpSolution)

    return tmpSolution

SPTSol = LongestProcessingTime(data.InputJobs)
print(SPTSol)

The permutation [9, 5, 1, 6, 10, 4, 3, 0, 8, 7, 2] results in a Makespan of 10649


Erwarteter Output:

    The permutation [9, 5, 1, 6, 10, 4, 3, 0, 8, 7, 2] results in a Makespan of 10649

#### d.) NEH Heuristik
Auch wenn Sie durch die Dispatching Rules sehr einfach und schnell Lösungen generieren können, sind Sie doch von der Lösungsgüte etwas enttäuscht. Selbst die SPT Regel führt zu fast 10% Abweichung von der optimalen Lösung. Aus diesem Grund erachten Sie es als sinnvoll, aufwendigere konstruktive Verfahren zu analysieren. Schnell stellen Sie fest, dass die NEH Heuristik zu den am meisten verwendeten Ansätzen gehört und häufig sehr gute Startlösungen liefert. Deshalb wollen Sie im Folgenden selbst eine Methode **NEH()** schreiben und das Verfahren implementieren.

In [11]:
from copy import deepcopy

def DetermineBestInsertion(solution, jobToInsert):
    ###
    # insert job at front of permutation
    solution.Permutation.insert(0, jobToInsert)  # insert函数，在索引位置插入数据，索引后面的往后顺移一位。初始肯定在第0位插入
    bestPermutation = deepcopy(solution.Permutation)
    
    EvaluationLogic().DefineStartEnd(solution)
    bestCmax = solution.Makespan

    ###
    # swap job i to each position and check for improvement
    lengthPermutation = len(solution.Permutation) - 1 #减去1是为了得到使用insert时的索引号
    for j in range(0, lengthPermutation):
        solution.Permutation[j], solution.Permutation[j + 1] = solution.Permutation[j+1], solution.Permutation[j]
        EvaluationLogic().DefineStartEnd(solution)
        if(solution.Makespan < bestCmax):
            bestCmax = solution.Makespan
            bestPermutation = [x for x in solution.Permutation]

    solution.Makespan = bestCmax
    solution.Permutation = bestPermutation

def NEH(jobList):
    jobPool = []
    tmpPerm = []
    bestCmax = 0
    # Calculate sum of processing times and sort
    for i in range(len(jobList)):
        jobPool.append((i,sum(jobList[i].ProcessingTime(x) for x in range(len(jobList[i].Operations)))))
    jobPool.sort(key=lambda x: x[1], reverse=True)

    # Initalize input
    tmpNEHOrder = [x[0] for x in jobPool]
    tmpPerm.append(tmpNEHOrder[0])
    tmpSolution = Solution(jobList,tmpPerm)

    # Add next jobs in a loop and check all permutations
    for i in range(1,len(tmpNEHOrder)):
        # add next job to end and calculate makespan
        DetermineBestInsertion(tmpSolution, tmpNEHOrder[i])
    
    return tmpSolution

NEHSol = NEH(data.InputJobs)    
print(NEHSol)



The permutation [7, 0, 4, 8, 2, 10, 3, 6, 5, 1, 9] results in a Makespan of 7038


Erwarteter Output:

    The permutation [7, 0, 4, 8, 2, 10, 3, 6, 5, 1, 9] results in a Makespan of 7038

#### e.)

Glücklich darüber, dass Sie mit Hilfe der NEH eine sehr gute Lösung gefunden haben. Wollen Sie Ihre implementierten konstruktiven Lösungsansätze abschließend in einer eigenen Klasse zusammenfassen. Nennen Sie die neue Klasse **ConstructiveHeuristic**. Damit Sie zukünftig einfach auf die einzelnen Methoden zugreifen können, entscheiden Sie sich, eine Hauptroutine zu schreiben, über welche Sie unter Angabe der jeweiligen konstruktiven Heuristik auf die Methoden zugreifen können.

In [13]:
class ConstructiveHeuristics:
    def Run(self, inputData, solutionMethod):
        if solutionMethod == 'FCFS':
            solution = FirstComeFirstServe(inputData.InputJobs)
        elif solutionMethod == 'SPT':
            solution = ShortestProcessingTime(inputData.InputJobs)
        elif solutionMethod == 'LPT':
            solution = LongestProcessingTime(inputData.InputJobs)
        elif solutionMethod == 'ROS':
            solution = ROS(inputData.InputJobs, 323, 10)
        elif solutionMethod == 'NEH':
            solution = NEH(inputData.InputJobs)
        else:
            print('Unkown constructive solution method.')    
        return solution

print(ConstructiveHeuristics().Run(data,"NEH"))

The permutation [7, 0, 4, 8, 2, 10, 3, 6, 5, 1, 9] results in a Makespan of 7038


---
### 3. Liefertreue 
--- 
Ihre bisherigen Lösungsversuchen konzentrieren sich auf eine möglichst effiziente Fertigung, indem die Kapazitäten möglichst gut ausgelastet werden. Allerdings stellt für Dr. Best auch die Liefertreue ein wichtiges Kriterium dar, damit die Kunden immer pünklich ihre Zähne putzen können. Deshalb soll im Folgenden **Total Tardiness** als alternatives Zielkriterium betrachtet werden.

a) 
Welche gesamten Verspätungen ergeben sich mit den bisherigen konstruktiven Lösungsansätzen **NEH**, **SPT** und **FCFS**? Schreiben Sie zuvor zwei Methoden in der Klasse EvaluationLogic, welche Total Tardiness und Total Weighted Tardiness berechnen.

In [14]:
def CalculateTardiness(self, currentSolution):
        totalTardiness = 0
        for key in currentSolution.OutputJobs:
            if(currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate > 0):
                currentSolution.OutputJobs[key].Tardiness = currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate
                totalTardiness += currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate
        currentSolution.TotalTardiness = totalTardiness       

setattr(EvaluationLogic,"CalculateTardiness",CalculateTardiness)

def CalculateWeightedTardiness(self, currentSolution):
    totalWeightedTardiness = 0
    for key in currentSolution.OutputJobs:
        if(currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate > 0):
            currentSolution.OutputJobs[key].Tardiness = currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate
            totalWeightedTardiness += (currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate) * currentSolution.OutputJobs[key].TardCost
    currentSolution.TotalWeightedTardiness = totalWeightedTardiness

setattr(EvaluationLogic,"CalculateWeightedTardiness",CalculateWeightedTardiness)


EvaluationLogic().CalculateTardiness(NEHSol)
print("NEH Regel führt zu gesamten Verspätungen von: "+ str(NEHSol.TotalTardiness))
EvaluationLogic().CalculateTardiness(FCFSSol)
print("FCFS Regel führt zu gesamten Verspätungen von: "+ str(FCFSSol.TotalTardiness))
EvaluationLogic().CalculateTardiness(SPTSol)
print("SPT Regel führt zu gesamten Verspätungen von: "+ str(SPTSol.TotalTardiness))

NEH Regel führt zu gesamten Verspätungen von: 18152
FCFS Regel führt zu gesamten Verspätungen von: 26812
SPT Regel führt zu gesamten Verspätungen von: 40081


Erwarteter Output:

    NEH Regel führt zu gesamten Verspätungen von: 18152
    FCFS Regel führt zu gesamten Verspätungen von: 26812
    SPT Regel führt zu gesamten Verspätungen von: 21866

#### b)
Da die bisherigen konstruktiven Lösungsansätze noch nicht auf Tardiness ausgerichtet sind, vermuten Sie, dass noch Potential zur Verbesserung besteht. Deshalb wollen Sie im Folgenden die Entscheidungsregel **Earliest Due Date** zur Lösungserzeugung nutzen. Schreiben Sie dafür eine entsprechende Funktion. Wie hoch sind hierbei die Verspätungen?

In [16]:
def EarliestDueDate(jobList):
    tmpPermutation = sorted(range(len(jobList)), key=lambda x: jobList[x].DueDate)
    tmpSolution = Solution(jobList, tmpPermutation)
    EvaluationLogic().DefineStartEnd(tmpSolution)
    EvaluationLogic().CalculateTardiness(tmpSolution)
    return(tmpSolution) 

EDDSol = EarliestDueDate(data.InputJobs)    
print("EDD Regel führt zu gesamten Verspätungen von: "+ str(EDDSol.TotalTardiness))

EDD Regel führt zu gesamten Verspätungen von: 15859


Erwarteter Output:
   
    EDD Regel führt zu gesamten Verspätungen von: 15859